## Gitcoin Grants #14 Trust & DeSoc Scores (🤖,💚)

# Part 2: Processing


This notebook processes data from Gitcoin Grants and on-chain sources.

### Dependencies

In [ ]:
from collections import Counter
import json
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer

from settings import PATHS

# 1/ Ingest GR 14 data

This requires access to a private dataset from Gitcoin and on-chain queries, which were extracted in Part 1. 

Now we load the "clean" dataset for additional processing.

In [ ]:
gr14_df = pd.read_csv(PATHS['cleancsv'], index_col='address')
gr14_df.head(2)

In [ ]:
wallets = list(gr14_df.index)
len(wallets)

## Quick analysis of Passport trust scores vs other indicators

In [ ]:
# create 4 bins for trust groups
trust_groups = pd.cut(gr14_df['trustscore'], 4).astype(str)
gr14_df['binTrustscore'] = trust_groups
(gr14_df
     .groupby('binTrustscore')
     [['isSquelched', 'qfScore', 'sumUSD', 'numGrants', 'numDonations']]
     .mean()
     .join(
         trust_groups
         .value_counts()
         .rename('countAddresses')
     )
).T

# 2/ Process on-chain activity

Create some simple DeSoc metrics for each on-chain data source.

Reminder of where the data come from:

- POAPs (xDAI only) // [api](https://api.thegraph.com/subgraphs/name/poap-xyz/poap-xdai)
- Snapshot votes (all EVM chains) // [api](https://hub.snapshot.org/graphql/)
- NFT ownership (ETH Mainnet only) // [api](https://api.zora.co/graphql)
- Lens protocol // [api](https://api.lens.dev/)
- Proof of Humanity // [api](https://api.poh.dev)

## Lens Protocol

In [ ]:
with open(PATHS['graphs']['lens'], 'r') as infile:
    json_data = json.load(infile)
lens_df = pd.DataFrame(json_data['data'])

lens_df['ownedBy'] = lens_df['ownedBy'].str.lower()
lens_df['numLens'] = 1
lens_df['numLensFollowers'] = lens_df['stats'].apply(lambda x: x['totalFollowers'])
lens_df['numLensFollowing'] = lens_df['stats'].apply(lambda x: x['totalFollowing'])
lens_df = (lens_df
           .drop(columns=['stats', 'id', 'name'])
           .groupby('ownedBy').sum())
lens_df.head()

In [ ]:
print("Num wallets with Lens profile:", len(lens_df))
print("Share of wallets on Lens:", len(lens_df) / len(wallets))

In [ ]:
# scatter plot of Lens Followings vs Followers

fig = px.scatter(
    data_frame=lens_df,
    x='numLensFollowers',
    y='numLensFollowing',
    size='numLens',
    opacity=.5,
)
fig.update_xaxes(range=(0,lens_df['numLensFollowers'].quantile(.99)),linecolor='black',)
fig.update_yaxes(range=(0,lens_df['numLensFollowing'].quantile(.99)), linecolor='black',)
fig.update_layout(plot_bgcolor='white')
fig.update_traces(marker_line_width=0)
fig

## POAP

View POAP events by changing the number at the end of the URL

https://poap.gallery/event/12345

In [ ]:
with open(PATHS['graphs']['poaps'], 'r') as infile:
    json_data = json.load(infile)
poap_df = pd.DataFrame(json_data['data'])

poap_df['numPOAPs'] = poap_df['tokensOwned'].astype(int)
poap_df['setPOAPs'] = poap_df['tokens'].apply(
    lambda events: 
    set([int(e['event']['id']) for e in events]) 
    if len(events) else []
)
poap_df.drop(columns=['tokensOwned', 'tokens'], inplace=True)
poap_df.set_index('id', inplace=True)
poap_df.head(2)

In [ ]:
print("Total POAPs collected:", poap_df['numPOAPs'].sum())
print("Num wallets with POAPs:", len(poap_df[poap_df['numPOAPs']>0]))
print("Share of wallets with POAPs", len(poap_df[poap_df['numPOAPs']>0]) / len(wallets))

In [ ]:
poap_df['numPOAPs'].describe()

In [ ]:
# Histogram of wallets with more than 100 POAPs

fig = px.histogram(
    data_frame=poap_df[poap_df['numPOAPs']>100], 
    x='numPOAPs'
)
fig

### Get a matrix of POAPs by addresses

In [ ]:
mlb = MultiLabelBinarizer()
poap_grid = pd.DataFrame(
    mlb.fit_transform(poap_df['setPOAPs']),
    columns=mlb.classes_, 
    index=poap_df.index
)
poap_grid.columns = [str(c) for c in poap_grid.columns]
poap_grid.shape

In [ ]:
# visualize the most collected POAPs among GR14 funders

poap_counts = poap_grid.sum().sort_values()
most_popular_poaps = poap_counts.tail(20)
fig = px.bar(most_popular_poaps, orientation='h')
fig.update_layout(
    plot_bgcolor='#FAFAFA',
    paper_bgcolor='#FAFAFA',
    font_family='Arial',
    font_size=18,
    showlegend=False,
    width=1200,
    height=600,
    yaxis=dict(
        linecolor='black',
        showgrid=False,
        zeroline=False,
    ),
    xaxis=dict(
        linecolor='black',
        showgrid=False,
        zeroline=False,
    )
)
fig.update_xaxes(title='# GR14 wallets')
fig.update_yaxes(title='POAP event #')
fig

In [ ]:
# have a look for potential POAP farmers

poap_farmers = poap_grid[most_popular_poaps.index].sum(axis=1).sort_values()
poap_farmers = poap_farmers[poap_farmers > 10].index

print(f"{len(poap_farmers)} wallets have more than 10 of the top 25 POAPs")

fig = px.density_heatmap(
    data_frame=(poap_grid
                .loc[poap_farmers, most_popular_poaps.index]
                .reset_index()
                .melt(id_vars='id', var_name='event')),
    x='event',
    y='id',
    z='value',
    color_continuous_scale='Greens'
)
fig.update_layout(coloraxis_showscale=False)
fig.update_yaxes(showticklabels=False, title='wallet addresses')
fig

### Simple POAP event correlation analysis

In [ ]:
c = (poap_grid
     [poap_counts[poap_counts>800].index]
     .corr())
s = c.unstack()
s = s[(s<1) & (s>.5)]
poap_watchlist = set([e for pair in s.index for e in pair])

fig,ax = plt.subplots(figsize=(15,10), dpi=144)
sns.heatmap(
    poap_grid[poap_watchlist].corr(),
    cmap='Purples',
    annot=True,
    fmt=".1f"
)

## Snapshot

In [ ]:
with open(PATHS['graphs']['snapshot'], 'r') as infile:
    json_data = json.load(infile)
snapshot_df = pd.DataFrame(json_data['data'])

snapshot_df['proposal'] = snapshot_df['proposal'].apply(lambda x: x['id'] if isinstance(x, dict) else None)
snapshot_df['space'] = snapshot_df['space'].apply(lambda x: x['id'])
snapshot_df['voter'] = snapshot_df['voter'].str.lower()

print("Number of votes recorded:", len(snapshot_df))
snapshot_df.head(2)

In [ ]:
voters_df = (
    pd.concat([
        snapshot_df
        .groupby('voter')['space']
        .apply(set)
        .rename("setSpaces"),
        snapshot_df
        .groupby('voter')['id']
        .count()
        .rename("numVotes")
    ], axis=1)
)
voters_df['numSpaces'] = voters_df['setSpaces'].apply(len)
voters_df.head(2)

In [ ]:
print("Num walets that vote on Snapshot", len(voters_df))
print("Share of wallets that vote on Snapshot", len(voters_df) / len(wallets))

In [ ]:
space_counts = (snapshot_df
                 .groupby('voter')
                 ['space']
                 .agg(lambda x: Counter(x))
                 .rename('spaceCounts'))
voter_df = voters_df.join(space_counts)
voter_df.head(2)

### Rankings by DAO / affiliation

In [ ]:
ranked_spaces = (snapshot_df
                 .groupby('space')['voter']
                 .agg(lambda x: len(x.unique()))
                 .sort_values())
spaces = sorted(ranked_spaces.index)
ranked_spaces.tail()

In [ ]:
# visualize the most popular DAO spaces

fig = px.bar(ranked_spaces.tail(25))
fig.update_layout(showlegend = False)
fig.update_xaxes(title='DAO space')
fig.update_yaxes(title='# GR14 wallets')
fig

In [ ]:
# visualize the distribution of the # of proposals voted on by DAO members

snapshot_votes_by_space = snapshot_df.groupby('space')['proposal'].agg(lambda x: len(x.unique()))
fig = px.histogram(snapshot_votes_by_space[snapshot_votes_by_space<10])
fig.update_layout(showlegend = False)
fig.update_xaxes(title='# proposals voted on in the DAO')
fig.update_yaxes(title='# of DAOs')
fig

In [ ]:
mlb = MultiLabelBinarizer()
space_grid = pd.DataFrame(
    mlb.fit_transform(voters_df['setSpaces']),
    columns=mlb.classes_, 
    index=voters_df.index
)
space_grid.shape

In [ ]:
most_popular_spaces = space_grid.sum().sort_values().tail(25)
daoers = space_grid[most_popular_spaces.index].sum(axis=1).sort_values()
daoers = daoers[daoers > 5].index

print(f"{len(daoers)} wallets have voted in more than 5 of the top 25 spaces")

fig = px.density_heatmap(
    data_frame=(space_grid
                .loc[daoers, reversed(most_popular_spaces.index)]
                .reset_index()
                .melt(id_vars='voter', var_name='space')),
    x='space',
    y='voter',
    z='value',
    color_continuous_scale='Greens'
)
fig.update_layout(coloraxis_showscale=False)
fig.update_yaxes(showticklabels=False)
fig

In [ ]:
# look for correlations in DAO affiliations

fig,ax = plt.subplots(figsize=(15,10), dpi=144)
sns.heatmap(
    space_grid.loc[daoers, most_popular_spaces.index].corr(),
    cmap='Purples',
    annot=True,
    fmt=".1f"
)

## Zora / NFTs
Pulls relevant NFT data (on ETH only). API has limit of 500 results per page, so this may not be an exhaustive list for heavy NFT collectors.

In [ ]:
from utils.zora import POPULAR_NFTS

In [ ]:
with open(PATHS['graphs']['nfts'], 'r') as infile:
    json_data = json.load(infile)
zora_df = pd.DataFrame([i['token'] for i in json_data['data']])
print("Total NFTs in dataset", len(zora_df))
zora_df.head(2)

In [ ]:
nft_df = (
    pd.concat([
        
        zora_df
        .groupby('owner')['collectionAddress']
        .apply(set)
        .rename("setCollections"),
        
        zora_df
        .groupby('owner')['collectionAddress']
        .agg(lambda x: len(x.unique()))
        .rename("numCollections"),
    
        zora_df
        .groupby('owner')['collectionAddress']
        .count()
        .rename("numNFTs")

    ], axis=1)
)
nft_df.head(2)

### Some analysis on NFT patterns

In [ ]:
print("Num wallets with NFTs", len(nft_df))
print("Share of wallets with NFTs", len(nft_df[nft_df['numNFTs']>0]) / len(wallets))

In [ ]:
# histogram of NFT counts

fig = px.histogram(
    data_frame=nft_df,
    x='numNFTs',
)
fig

### Rankings by NFT collection

In [ ]:
zora_df['collectionName'] = zora_df['collectionAddress'].replace(POPULAR_NFTS)
nft_df['setCollections'] = nft_df['setCollections'].apply(
    lambda x: set([POPULAR_NFTS.get(c,c) for c in x])
)

In [ ]:
collection_counter = (zora_df
                     .groupby('owner')
                     ['collectionName']
                     .agg(lambda x: Counter(x))
                     .rename('countNFTs'))
nft_df = nft_df.join(collection_counter) # todo: refactor
nft_df.head(2)

In [ ]:
# see which collections are most popular

ranked_collections = (zora_df
                      .groupby('collectionName')['owner']
                      .agg(lambda x: len(x.unique()))
                      .sort_values())
collections = sorted(ranked_collections.index)
ranked_collections.tail(15)

In [ ]:
# visualize the most popular NFT collections

fig = px.bar(
    ranked_collections[POPULAR_NFTS.values()].sort_values(),
    orientation='h'
)
fig.update_layout(showlegend = False)
fig.update_xaxes(title='# NFTs')
fig.update_yaxes(title='Collection')
fig

In [ ]:
# visualize correlations among different NFT collections

nft_grid = (zora_df[zora_df
                    .collectionName
                    .isin(list(POPULAR_NFTS.values()))
                   ]
            .groupby(['owner', 'collectionName'])['tokenId']
            .count()
            .rename('numTokens')
            .reset_index())
nft_grid['hasToken'] = nft_grid['numTokens'].apply(lambda x: min(1,x))

collectors = nft_grid.groupby('owner')['hasToken'].sum()
collectors = collectors[collectors>=8].index

print(f"{len(collectors)} wallets have at least one NFT from 8 of the top 15 collections")

fig = px.density_heatmap(
    data_frame=nft_grid[nft_grid['owner'].isin(collectors)],
    x='collectionName',
    y='owner',
    z='hasToken',
    color_continuous_scale='Greens',
)
fig.update_layout(coloraxis_showscale=False)
fig.update_yaxes(showticklabels=False)
fig

## Proof of Humanity

In [ ]:
with open(PATHS['graphs']['poh'], 'r') as infile:
    json_data = json.load(infile)
poh_wallets = set(json_data).intersection(set(wallets))
poh_series = pd.Series([True]*len(poh_wallets), index=poh_wallets, name='hasProofOfHumanity')


print("Num wallets on PoH", len(json_data))
print("Share of wallets that are on PoH", len(poh_series) / len(wallets))

# 3/ Consolidate GR14 + onchain data into a single data file

In [ ]:
df = (gr14_df
      .join(voters_df)
      .join(poap_df)
      .join(nft_df)
      .join(lens_df)
      .join(poh_series)
     )

# fill 0 for numerics with NaN 
numerics = df.select_dtypes(include=['number']).columns
df[numerics] = df[numerics].fillna(0)
df['hasProofOfHumanity'].fillna(False, inplace=True)

print("Total wallets:", len(df))
df.head(3)

In [ ]:
# dump into a pickle file
df.to_pickle(PATHS['outdata'])